![NYC Skyline](data/nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this project, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
This is a CSV file containing data on Airbnb listing prices and locations.
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

In [1]:
# Import necessary packages
import pandas as pd
import numpy as np

There are three files in the data folder: `airbnb_price.csv`, `airbnb_room_type.xlsx`, `airbnb_last_review.tsv`.

What are the dates of the earliest and most recent reviews? Store these values as two separate variables with your preferred names.
How many of the listings are private rooms? Save this into any variable.
What is the average listing price? Round to the nearest two decimal places and save into a variable.
Combine the new variables into one DataFrame called `review_dates` with four columns in the following order: `first_reviewed`, `last_reviewed`, `nb_private_rooms`, and `avg_price`. The DataFrame should only contain one row of values.

In [2]:
price = pd.read_csv("data/airbnb_price.csv")
room_type = pd.read_excel("data/airbnb_room_type.xlsx")
last_review = pd.read_csv("data/airbnb_last_review.tsv", sep="\t")

In [3]:
price.head()

,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"
3,5178,79 dollars,"Manhattan, Hell's Kitchen"
4,5238,150 dollars,"Manhattan, Chinatown"


In [4]:
room_type.head()

,listing_id,description,room_type
0,2595,Skylit Midtown Castle,Entire home/apt
1,3831,Cozy Entire Floor of Brownstone,Entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt
3,5178,Large Furnished Room Near B'way,private room
4,5238,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt


In [5]:
last_review.head()

,listing_id,host_name,last_review
0,2595,Jennifer,May 21 2019
1,3831,LisaRoxanne,July 05 2019
2,5099,Chris,June 22 2019
3,5178,Shunichi,June 24 2019
4,5238,Ben,June 09 2019


In [6]:
# Verify that each DataFrame has a unique, non-null 'listing_id'
for name, df in [('price', price), ('room_type', room_type), ('last_review', last_review)]:
    # Column exists
    assert "listing_id" in df.columns, f"{name!r} lacks column 'listing_id'"
    # No missing values
    assert not df['listing_id'].isnull().any(), f"Nulls detected in {name}['listing_id']"
    # Uniqueness
    assert df['listing_id'].is_unique, f"Duplicates detected in {name}['listing_id']"

print("Integrity checks passed for all three DataFrames.")

Integrity checks passed for all three DataFrames.


In [7]:
# Merge all three DataFrames into one complete dataset
# First merge price and room_type data on listing_id
# Then merge with last_review data on listing_id
# Using left joins to keep all records from the price DataFrame
# validate="one_to_one" ensures each listing_id appears only once in each DataFrame
airbnb_full = price.merge(
    room_type, on="listing_id", how="left", validate="one_to_one"
).merge(last_review, on="listing_id", how="left", validate="one_to_one")

In [8]:
# Clean the price column by removing "dollars" text and converting to integer
airbnb_full["price"] = airbnb_full["price"].str.replace("dollars", "").astype("int")

# Convert last_review column to datetime format, handling invalid dates gracefully
airbnb_full["last_review"] = pd.to_datetime(airbnb_full["last_review"], errors="coerce")

# Lower the room_type categories to ensure consistency
airbnb_full["room_type"] = airbnb_full["room_type"].str.lower().astype("category")
airbnb_full

,listing_id,price,nbhood_full,description,room_type,host_name,last_review
0,2595,225,"Manhattan, Midtown",Skylit Midtown Castle,entire home/apt,Jennifer,2019-05-21
1,3831,89,"Brooklyn, Clinton Hill",Cozy Entire Floor of Brownstone,entire home/apt,LisaRoxanne,2019-07-05
2,5099,200,"Manhattan, Murray Hill",Large Cozy 1 BR Apartment In Midtown East,entire home/apt,Chris,2019-06-22
3,5178,79,"Manhattan, Hell's Kitchen",Large Furnished Room Near B'way,private room,Shunichi,2019-06-24
4,5238,150,"Manhattan, Chinatown",Cute & Cozy Lower East Side 1 bdrm,entire home/apt,Ben,2019-06-09
...,...,...,...,...,...,...,...
25204,36425863,129,"Manhattan, Upper East Side",Lovely Privet Bedroom with Privet Restroom,private room,Rusaa,2019-07-07
25205,36427429,45,"Queens, Flushing",No.2 with queen size bed,private room,H Ai,2019-07-07
25206,36438336,235,"Staten Island, Great Kills",Seas The Moment,private room,Ben,2019-07-07
25207,36442252,100,"Bronx, Mott Haven",1B-1B apartment near by Metro,entire home/apt,Blaine,2019-07-07


In [9]:
# What are the dates of the earliest and most recent reviews? Store these values as two separate variables with your preferred names.
earliest = airbnb_full["last_review"].min()
most_recent = airbnb_full["last_review"].max()

# How many of the listings are private rooms? Save this into any variable.
private_rooms = airbnb_full[airbnb_full["room_type"] == "private room"].shape[0]

# What is the average listing price? Round to the nearest two decimal places and save into a variable.
average_price = airbnb_full["price"].mean()

In [ ]:
# Combine the new variables into one DataFrame called review_dates with four columns in the following order: first_reviewed, last_reviewed, nb_private_rooms, and avg_price. The DataFrame should only contain one row of values.
review_dates = pd.DataFrame({
    'first_reviewed': [earliest],
    'last_reviewed': [most_recent],
    'nb_private_rooms': [private_rooms],
    'avg_price': [round(average_price, 2)]
})
review_dates

,first_reviewed,last_reviewed,nb_private_rooms,avg_price
0,2019-01-01,2019-07-09,11356,141.78


: 